In [ ]:
import numpy as np

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
import cv2 as cv

In [ ]:
from sklearn import linear_model

In [ ]:
import warnings

In [ ]:
plt.style.use('seaborn')

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
def plot_image(image, i=0):

In [ ]:
    l = [cv.COLOR_BGR2RGB, cv.COLOR_BGR2GRAY, cv.COLOR_RGB2GRAY,]

In [ ]:
    plt.imshow(cv.cvtColor(image, l[i]))

In [ ]:
    plt.show(block=True)

In [ ]:
def display_image(image):

In [ ]:
    cv.imshow('image', image)

In [ ]:
    cv.waitKey(0)

In [ ]:
    cv.destroyAllWindows()

In [ ]:
def plot_graph(image, i=0):

In [ ]:
    plt.figure(figsize=(15, 8))

In [ ]:
    sns.scatterplot(np.arange(len(image.sum(i))), image.sum(i), label='a', alpha=0.6)

In [ ]:
def MinMaxScalar(x, a=-1, b=1):

In [ ]:
    N = (x - x.min()) * (b - a)

In [ ]:
    D = (x.max() - x.min())

In [ ]:
    return (N / D) + a

In [ ]:
def sigmoid(t):

In [ ]:
    return 1 / (1 + np.exp(-t))

In [ ]:
def softplus(t):

In [ ]:
    return np.log(1 + np.exp(t))

In [ ]:
def preprocess_image(image):

In [ ]:
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    blur = cv.bilateralFilter(gray, 3, 75, 75)
    thresh = cv.adaptiveThreshold(blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    k = 3
    kernal = np.ones((k, k), np.uint8)
    dilate = cv.dilate(thresh, kernal, iterations=1)
    k = 1
    kernal = np.ones((k, k), np.uint8)
    erode = cv.erode(dilate, kernal, iterations=1)
    return erode

In [ ]:
def line_extraction(preprocess_imaged, r_image):

In [ ]:
    gray = cv.cvtColor(r_image, cv.COLOR_BGR2GRAY)
    edges = cv.Canny(preprocess_imaged, 0, 125, apertureSize=3)
    lines = cv.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=int(r_image.shape[1] * 0.0002))
    Y = [0,]
    for line in lines:
        x1, y1, x2, y2 = line[0]
        Y.append(y1)
    Y.append(gray.shape[0])
    Y = sorted(Y)
    Y_m = []
    for i in range(len(Y)):
        if i == len(Y)-1:
            break
        Y_m.append(Y[i + 1] - Y[i])
    LR = linear_model.LinearRegression()
    LR.fit(np.array(range(len(Y_m))).reshape(-1, 1), np.array(Y_m))
    m = LR.coef_ 
    c = LR.intercept_
    Y_bool = (np.array(Y_m) - (m * np.array(range(len(Y_m)))) - c) > 0
    Y_bool = np.where(Y_bool>0)[0]
    line_extractions = []
    for i in range(len(Y_bool)):
        line_extractions.append(r_image[:][Y[Y_bool[i]] : Y[Y_bool[i] + 1]])
    return line_extractions

In [ ]:
def preprocess_line(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    blur = cv.bilateralFilter(gray, 3, 75, 75)
    thresh = cv.adaptiveThreshold(blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    k = 5
    kernal = np.ones((k, k), np.uint8)
    dilate = cv.dilate(thresh, kernal, iterations=1)
    k = 21
    kernal = np.ones((k, k), np.uint8)
    erode = cv.erode(dilate, kernal, iterations=1)
    return erode

In [ ]:
def word_extraction(preprocessed_line1, line1):
    sum_line = preprocessed_line1.sum(axis=0)
    sum_line_scaled = MinMaxScalar(sum_line, -1, 1)
    scaled_activated_sum = softplus(sigmoid(sum_line_scaled))
    l = scaled_activated_sum
    maximum = max(l)
    cutout_dip = []
    cutout_up = []
    for i in range(len(l)):
        if (i + 1) != len(l)):
            if l[i] == maximum:
                if (l[i + 1] < maximum):
                    cutout_dip.append(i)
            if l[i] < maximum):
                if (l[i + 1] == maximum):
                    cutout_up.append(i)
    cutout_images = []
    gray_line1 = cv.cvtColor(line1, cv.COLOR_BGR2GRAY)
    for up, dip in zip(cutout_up, cutout_dip):
        temp = gray_line1[:, dip : up]
        cutout_images.append(temp)
    return cutout_images

In [ ]:
real_page = cv.imread('page03.jpg')

In [ ]:
plot_image(real_page)

In [ ]:
preprocess_page = preprocess_image(real_page)

In [ ]:
line_extracted = line_extraction(preprocess_page, real_page)

In [ ]:
for line in line_extracted:
    line_p = cv.copyMakeBorder(line, 0, 0, 20, 20, cv.BORDER_CONSTANT, value=(255, 255, 255))
    preprocessed_line = preprocess_line(line)
    preprocessed_line_p = cv.copyMakeBorder(preprocessed_line, 0, 0, 20, 20, cv.BORDER_CONSTANT, value=255)
    word_extracted = word_extraction(preprocessed_line_p, line_p)
    for word in word_extracted:
        print('words')
        word_p = cv.copyMakeBorder(word, 10, 10, 0, 0, cv.BORDER_CONSTANT, value=255)
        preprocessed_word = preprocess_word(word_p)
        word_t = preprocessed_word.T
        word_r = word_extraction_T(word_t, word_p.T)
        for word_ in word_r:
            plot_image(word_)

In [ ]:
wanted, unwanted = word_extracted[-2], word_extracted[-1]

In [ ]:
plot_image(wanted)

In [ ]:
plot_image(unwanted)

In [ ]:
plot_graph(wanted)

In [ ]:
plot_graph(unwanted)

In [ ]:
wanted.sum(axis=1).mean()

In [ ]:
unwanted.sum(axis=1).mean()

In [ ]:
jkl = wanted.sum(axis=1)
arr = []
for llp in range(0,len(jkl)-1,2):
    print(jkl[llp+1]-jkl[llp])
arr